In [ ]:
from llama_index.llms import Ollama
from llama_index.llms import ChatMessage
from llama_index.vector_stores.postgres import PGVectorStore
import textwrap

In [ ]:
llm = Ollama(base_url="http://192.168.1.42:11434", model="mistral", request_timeout=30.0)

In [ ]:
messages = [
    ChatMessage(
        role="system", content="You are a developer with a colorful personality"
    ),
    ChatMessage(role="user", content="What is my name"),
]
resp = llm.stream_chat(messages)

for r in resp:
    print(r.delta, end="")

In [20]:
from sqlalchemy import make_url
from dotenv import load_dotenv
import os
import psycopg2

load_dotenv()  # Load environment variables from .env file

# Get the environment variables
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')

# Construct the connection string
connection_string = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
url = make_url(connection_string)

vector_store = PGVectorStore.from_params(
    database=db_name,
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name="feature",
    embed_dim=1024,  # openai embedding dimension
)

# storage_context = StorageContext.from_defaults(vector_store=vector_store)
# index = VectorStoreIndex.from_documents(
#     documents, storage_context=storage_context, show_progress=True
# )
# query_engine = index.as_query_engine()

TypeError: <class 'sqlalchemy.sql._typing._HasClauseElement'> is not a generic class